# biopython
- [biopython中文文档-2020年版本](https://biopython-cn.readthedocs.io/zh_CN/latest/cn/chr02.html)
- [biopython官方文档](https://biopython.org/wiki/Documentation)

可以干什么？

1. 读入并解析各类序列数据或者比对结果。用于后续的操作
2. 连接生物学相关的数据库

可以干很多[Cool things](http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec370)

## 序列数据处理
### 序列数据的读入
常见的序列文件的存储格式有：

- fasta
- GenBank

在`Biopython`中可以通过`Bio.SeqIO`来读入并用于后续处理。[关于可以支持解析的数据](https://biopython.org/wiki/SeqIO)

而对于序列比对的结果则可以通过`Bio.AlignIO`来读入并用于后续处理


### `SeqIO`读取数据

`Bio.SeqIO.parse`读入的数据是一个**迭代器**`Bio.SeqIO.FastaIO.FastaIterator`类型。

如果需要全部读入可以通过使用`list()`来获取

`SeqIO.read("xx.fasta", "fasta")`用于读入一条序列

`SeqIO.to_dict()` 可以把数据转化成字典类型

In [2]:
# 读入fasta数据
from Bio import SeqIO
records = SeqIO.parse("Cypripedium_japonicum.fasta","fasta")
print(f"解析后的数据的类型：{type(records)}")

#循环读取
for record in SeqIO.parse("Cypripedium_japonicum.fasta","fasta"):
    print(record.id)
    print(record.seq)
    break

#一次性读入
records = list(SeqIO.parse("Cypripedium_japonicum.fasta","fasta"))
print(records[0].id)
print(records[0].seq)

解析后的数据的类型：<class 'Bio.SeqIO.FastaIO.FastaIterator'>
gi|350999290|gb|JF797156.1|
CGGCTTATATAGGTGAATCCATGGAGGGTCATCATTGACTAACTTTCGAAATAGTTTTTTTTAAGCTTATCCCAATTCAAGCAATGCAGGGTTCAATATAATCCACAGGGTTGTAGAAGAAAATGCAAATAGCTACAAATATGTATCTATTGTATATATGAATAAAGATAGATGATATTGCAGAGTTTGGAATAGAAAGAAGGGTTGGGGGGTCCTACTATATATATGTAATGTCTATGAGTATCAGTCCTTGTGTATGTTTCGAAGAATGGTTCCAGAATACGATTTAATAGTTAATAGAATAAAAAGTGCAGGTGGTTTACGTTATGGAAGAAACAAAAGTATATGTTATTTACTAGTTAGATAGGAATTATCCCTATCTCTTTTTTTCTAGCCAACTTCATTTATATGGATTACAATATCAGTCCTTTTTCTATTTTTTCTGTGATTGTTAATTGAATGAAAGAATAGAAGAGTTTATAAACAAGAAAACACAATGACTAAAACCGTTTACATAAAACTCCATCATGGGTAGAAAGAAAGGAAAAACGGTATACGGTATATGGAAGTAGTTCTGAAAATTCAATAATATTCTGTTGGAGTTTTTAGCTACTTCGACCCGATAGATTTTAGTGATAAGGGTCAATCAAAAAGAAAATAAATAAAAAAATAAGTAAGTAAAATAAGTTTAAGTAAAGTAAAAAAGTAGTGTAGTAAAGTAAATAGTCAAAGTAAAAGTAGAAGTATAAAAAGAAGTAGATAAAGATTAAGTAGTAATTCATTGGTTGGTTGTATCATTAACCATTTCTTTCTTTTGGACGAGGAAAT
gi|350999290|gb|JF797156.1|
CGGCTTATATAGGTGAATCCATGGAGGGTCATCATTGACTAACTTTCGAAATAGTTTTTTTTA

`SeqIO.to_dict()`可以把读入的迭代器转成dict类型，key是序列的id

In [3]:
#转化成字典

seq_dict = SeqIO.to_dict(SeqIO.parse("Cypripedium_japonicum.fasta","fasta"))
#把name作为key
seq_dict_name = SeqIO.to_dict(SeqIO.parse("Cypripedium_japonicum.fasta","fasta"),lambda rec : rec.name)



- 读入gzip压缩的数据


In [4]:
import gzip
from Bio import SeqIO
with gzip.open("Cypripedium_japonicum.fasta.gz", "rt") as handle:
    records = list(SeqIO.parse(handle, "fasta"))
    
print(records[0].id)

gi|350999290|gb|JF797156.1|


除了gzip压缩的数据。还可以读入更多压缩格式类型。

关于如何[读取数据和写入数据参阅](https://biopython-cn.readthedocs.io/zh_CN/latest/cn/chr05.html)

## 序列

Biopython读入后得到的序列类型是`Bio.SeqRecord.SeqRecord`。相关的[api](https://biopython.org/docs/latest/api/Bio.SeqRecord.html)

该类型下面可以调用的属性有
- `id`
- `seq`，得到的是`Bio.Seq.Seq`类型。 可以对它进行切片、索引。它具备`str`类型的绝大多数函数。**该类型是不可以修改其中的字符的**。并且具备一个字母表来说明这个序列是DNA还是RNA或者蛋白质。
- `name`
- `description`
- 等

可以直接通过调用属性的方式调取

In [5]:
from Bio.Seq import Seq
seq = list(SeqIO.parse("Cypripedium_japonicum.fasta","fasta"))[0]
print(type(seq))
print(seq.id)
print(seq.seq)


<class 'Bio.SeqRecord.SeqRecord'>
gi|350999290|gb|JF797156.1|
CGGCTTATATAGGTGAATCCATGGAGGGTCATCATTGACTAACTTTCGAAATAGTTTTTTTTAAGCTTATCCCAATTCAAGCAATGCAGGGTTCAATATAATCCACAGGGTTGTAGAAGAAAATGCAAATAGCTACAAATATGTATCTATTGTATATATGAATAAAGATAGATGATATTGCAGAGTTTGGAATAGAAAGAAGGGTTGGGGGGTCCTACTATATATATGTAATGTCTATGAGTATCAGTCCTTGTGTATGTTTCGAAGAATGGTTCCAGAATACGATTTAATAGTTAATAGAATAAAAAGTGCAGGTGGTTTACGTTATGGAAGAAACAAAAGTATATGTTATTTACTAGTTAGATAGGAATTATCCCTATCTCTTTTTTTCTAGCCAACTTCATTTATATGGATTACAATATCAGTCCTTTTTCTATTTTTTCTGTGATTGTTAATTGAATGAAAGAATAGAAGAGTTTATAAACAAGAAAACACAATGACTAAAACCGTTTACATAAAACTCCATCATGGGTAGAAAGAAAGGAAAAACGGTATACGGTATATGGAAGTAGTTCTGAAAATTCAATAATATTCTGTTGGAGTTTTTAGCTACTTCGACCCGATAGATTTTAGTGATAAGGGTCAATCAAAAAGAAAATAAATAAAAAAATAAGTAAGTAAAATAAGTTTAAGTAAAGTAAAAAAGTAGTGTAGTAAAGTAAATAGTCAAAGTAAAAGTAGAAGTATAAAAAGAAGTAGATAAAGATTAAGTAGTAATTCATTGGTTGGTTGTATCATTAACCATTTCTTTCTTTTGGACGAGGAAAT


### 序列数据处理

常用的一些函数用来计算含量、反向互补序列、计算编码链对应的蛋白质序列等

In [7]:
seq = Seq("ATACTAATAC")

#可以计算反向互补序列
complement = seq.complement()
print(complement)
reverse_complement = complement[::-1]
print(reverse_complement)

#直接调用
reverse_complement = seq.reverse_complement()
print(reverse_complement)

C_content = seq.count("C")
print(C_content)

#DNA转录和翻译的序列结果
rna_seq = seq.transcribe()
protein_seq = seq.translate()
print(rna_seq)
print(protein_seq)

#RNA转成对应的编码链的序列
print(rna_seq.back_transcribe())


TATGATTATG
GTATTAGTAT
GTATTAGTAT
2
AUACUAAUAC
ILI
ATACTAATAC


在使用的时候需要注意到模板链和编码链的区别

在biopython中，我们需要搞明白我们手中的序列是编码链还是模板链

默认序列从左到右是5到3端的

`transcribe()`只是把给定的序列中的t变成u，因此如果给定的序列是编码链，直接调用`transcribe()`就可以。而对于模板链则需要通过`reverse_complement()`获取对应的编码链的序列才行

RNA也可以用`back_transcribe()`获得对应的编码序列，其实就是把U换成T。

In [4]:
#编码链
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
#获取对应编码的rna
rna = coding_dna.transcribe()
print(f"编码链：{coding_dna} \n转录后的rna：{rna}")

print("如果给的是模板链")

#给的模板链，仍然是从5端到3端，从这里转录获得对应的编码链
template_dna = coding_dna.reverse_complement()
rna = template_dna.reverse_complement().transcribe()
print(f"模板链：{template_dna}\n编码的RNA：{rna}")

#如果给的是mRNA，也可以获得对应的序列
print(f"RNA：{rna}")
print(f"coding：{rna.back_transcribe()}")

编码链：ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG 
转录后的rna：AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
如果给的是模板链
模板链：CTATCGGGCACCCTTTCAGCGGCCCATTACAATGGCCAT
编码的RNA：AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
RNA：AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
coding：ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG


除了可以进行"转录"，还可以进行“翻译”

`translate()`方法可以获取对应的编码链编码的蛋白质序列。

- 可以通过添加Table，来指定使用的翻译表，也就是遗传密码。`Table = 2`表示使用来自[NCBI的遗传密码表](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi)中的第二个，也可以指定名称

In [5]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
print(f"编码链：{coding_dna}")
protein = coding_dna.translate()
print(f"直接获取翻译的蛋白质序列：{protein}")
#可以看到有两个终止符在里面
#如果我们告诉它这个序列是线粒体的序列，使用线粒体中常用的一套遗传密码
protein = coding_dna.translate(table="Vertebrate Mitochondrial")
print(f"使用线粒体中的遗传密码表获得的蛋白质序列：{protein}")

protein = coding_dna.translate(to_stop=True)
print(f"遇到第一个密码子就停止翻译的蛋白质序列，不会输出终止符：{protein}")

protein = coding_dna.translate(stop_symbol="&")
print(f"自定义终止符进行翻译：{protein}")

编码链：ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
直接获取翻译的蛋白质序列：MAIVMGR*KGAR*
使用线粒体中的遗传密码表获得的蛋白质序列：MAIVMGRWKGAR*
遇到第一个密码子就停止翻译的蛋白质序列，不会输出终止符：MAIVMGR
自定义终止符进行翻译：MAIVMGR&KGAR&


In [6]:
# 当然也可以获取这些表格
from Bio.Data import CodonTable

standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]
print(mito_table)

Table 2 Vertebrate Mitochondrial, SGC1

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA W   | A
T | TTG L   | TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L   | CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I(s)| ACT T   | AAT N   | AGT S   | T
A | ATC I(s)| ACC T   | AAC N   | AGC S   | C
A | ATA M(s)| ACA T   | AAA K   | AGA Stop| A
A | ATG M(s)| ACG T   | AAG K   | AGG Stop| G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V(s)| GCG A   | GAG E   | GGG G   

In [7]:
#也可以通过索引获得
from Bio.Data import CodonTable
standard_table = CodonTable.unambiguous_dna_by_id[1]
mito_table = CodonTable.unambiguous_dna_by_id[2]

In [8]:
print(f"Vertebrate Mitochondrial中的终止密码子：{mito_table.stop_codons}")
print(f"Vertebrate Mitochondrial中的起始密码子：{mito_table.start_codons}")
#查阅对应的aa和nucleotide的表
print(f"Vertebrate Mitochondrial中的碱基类型：{mito_table.nucleotide_alphabet}")
print(f"Vertebrate Mitochondrial中翻译产生的氨基酸表：{mito_table.protein_alphabet}")
#也可以直接根据密码子去查询对应的氨基酸，如TTT编码F，苯丙氨酸
## forward_table表示正向查询密码子编码的氨基酸
TTT = mito_table.forward_table["TTT"]
print(f"TTT密码子编码{TTT}")
F = mito_table.back_table["F"]
print(f"编码F即苯丙氨酸的密码子有：{F}")

Vertebrate Mitochondrial中的终止密码子：['TAA', 'TAG', 'AGA', 'AGG']
Vertebrate Mitochondrial中的起始密码子：['ATT', 'ATC', 'ATA', 'ATG', 'GTG']
Vertebrate Mitochondrial中的碱基类型：GATC
Vertebrate Mitochondrial中翻译产生的氨基酸表：ACDEFGHIKLMNPQRSTVWY
TTT密码子编码F
编码F即苯丙氨酸的密码子有：TTT


### `MutableSeq` 对象

Seq 对象是 “只读的” ，在Python术语上就是不可变的。 除了想要 Seq 对象表现得向一个字符串之外，这是一个很有用的默认，因为在生 物学应用上你往往需要确保你没有改动你的序列数据

In [9]:
seq = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
seq[5] = "T"

TypeError: 'Seq' object does not support item assignment

`MutableSeq`可以把一个`Seq`转换成可变的序列

`MutableSeq()`可以实现这一转换！

`Seq()`可以把它转回去

In [10]:
from Bio.Seq import MutableSeq

mutable_seq = MutableSeq(seq)
mutable_seq

MutableSeq('ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG')

In [11]:
print(mutable_seq[5])
mutable_seq[5] = "T"
print(mutable_seq[5])

C
T


In [12]:
#转成Seq类型
isinstance(Seq(mutable_seq), Seq)

True

### 未知字符的表示

用`Seq(None, length)`来产生一个代表含有位置字符的类型

In [13]:
unk = Seq(None, length=20)


### SeqRecord类型更多介绍

在前面已经介绍了许多关于SeqRecord的内容。它可以帮助我们更好的读取其中的序列信息和注释等内容。

其主要属性有以下

In [14]:
from Bio import SeqIO
record = next(SeqIO.parse("Cypripedium_japonicum.fasta","fasta"))
print(record)

print(record.id)
print(record.name)
print(record.description)

ID: gi|350999290|gb|JF797156.1|
Name: gi|350999290|gb|JF797156.1|
Description: gi|350999290|gb|JF797156.1| Cypripedium japonicum isolate IH43 atpI-atpH intergenic spacer, partial sequence; chloroplast
Number of features: 0
Seq('CGGCTTATATAGGTGAATCCATGGAGGGTCATCATTGACTAACTTTCGAAATAG...AAT')
gi|350999290|gb|JF797156.1|
gi|350999290|gb|JF797156.1|
gi|350999290|gb|JF797156.1| Cypripedium japonicum isolate IH43 atpI-atpH intergenic spacer, partial sequence; chloroplast


In [20]:
#读取genbank格式的数据
from Bio import SeqIO
record = next(SeqIO.parse("Cypripedium_japonicum.gb","genbank"))
# print(record)

print(record.id)
print(record.name)
print(record.description)
print(record.dbxrefs)
print(record.annotations)
print(len(record.features))#记录了序列中的一些特征，比如存在外显子，位于那个区域等信息

JF797156.1
JF797156
Cypripedium japonicum isolate IH43 atpI-atpH intergenic spacer, partial sequence; chloroplast
[]
2
